In [51]:
import pandas as pd
import os


In [62]:
#import .csvs
reg_season_wins = pd.read_csv('nfl_reg_season_win_loss_records_2011_to_2022.csv')
playoff_wins = pd.read_csv('nfl_playoff_win_loss_records_2011_to_2022.csv')
superbowl_wins = pd.read_csv('Super_Bowl_Winners.csv')
player_salaries = pd.read_csv('nfl_salaries.csv')

In [53]:
reg_season_wins

,Year,Team,Win,Loss,Tie
0,2011,Green Bay,15,1,0
1,2011,San Francisco,13,3,0
2,2011,New England,13,3,0
3,2011,New Orleans,13,3,0
4,2011,Baltimore,12,4,0
...,...,...,...,...,...
379,2022,LA Rams,5,12,0
380,2022,Indianapolis,4,12,1
381,2022,Arizona,4,13,0
382,2022,Houston,3,13,1


In [63]:
playoff_wins

,Year,Team,Win,Loss,Tie
0,2011,NY Giants,4,0,0
1,2011,New England,2,1,0
2,2011,Baltimore,1,1,0
3,2011,Houston,1,1,0
4,2011,Denver,1,1,0
...,...,...,...,...,...
145,2022,Minnesota,0,1,0
146,2022,Baltimore,0,1,0
147,2022,LA Chargers,0,1,0
148,2022,Seattle,0,1,0


In [55]:
superbowl_wins

,2011,NY Giants,Yes
0,2012,Baltimore,Yes
1,2013,Seattle,Yes
2,2014,New England,Yes
3,2015,Denver,Yes
4,2016,New England,Yes
5,2017,Philadelphia,Yes
6,2018,New England,Yes
7,2019,Kansas City,Yes
8,2020,Tampa Bay,Yes
9,2021,Los Angeles,Yes


In [56]:
player_salaries.head(60)

,index,year,team,name,position,cap_hit,base_salary,cap_percentage
0,1,2020,arizona-cardinals,Patrick Peterson,CB,"$12,050,000",-,6.71
1,2,2020,arizona-cardinals,Larry Fitzgerald,WR,"$11,000,000",-,5.93
2,3,2020,arizona-cardinals,Jordan Hicks,ILB,"$6,000,000","$3,000,000",5.34
3,4,2020,arizona-cardinals,Justin Pugh,G,"$7,250,000","$2,000,000",5.30
4,5,2020,arizona-cardinals,Kenyan Drake,RB,"$8,483,000",-,4.32
5,6,2020,arizona-cardinals,Kyler Murray,QB,"$675,000","$5,897,481",4.11
6,7,2020,arizona-cardinals,DeAndre Hopkins,WR,"$1,500,000","$5,500,000",3.56
7,8,2020,arizona-cardinals,J.R. Sweezy,G,"$3,500,000","$1,500,000",3.26
8,9,2020,arizona-cardinals,D.J. Humphries,LT,"$910,000","$800,000",2.92
9,10,2020,arizona-cardinals,Haason Reddick,ILB,"$2,302,904","$1,985,540",2.18


# First analysis: Grab the highest paid player (by percent of cap), bucket them into groups, and determine if theres correlation between the highest paid player and success

In [57]:
#delete this cell when you have clean data
player_salaries = player_salaries[player_salaries['year'] != 2023]
player_salaries.tail(60)

,index,year,team,name,position,cap_hit,base_salary,cap_percentage
5119,5120,2022,tennessee-titans,Daniel Munyer,C,"$965,000",-,0.08
5120,5121,2022,tennessee-titans,Julius Chestnut,RB,"$705,000",-,0.07
5121,5122,2022,tennessee-titans,Joshua Dobbs,QB,"$1,035,000",-,0.07
5122,5123,2022,tennessee-titans,Xavier Newman-Johnson,G,"$705,000",-,0.06
5123,5124,2022,tennessee-titans,Andre Smith,ILB,"$1,035,000",-,0.05
5124,5125,2022,tennessee-titans,Jonathan Ward,RB,"$895,000",-,0.05
5125,5126,2022,tennessee-titans,Mike Brown,S,"$705,000",-,0.02
5126,5127,2022,washington-football-team,Carson Wentz,QB,"$22,000,001",-,13.22
5127,5128,2022,washington-football-team,Curtis Samuel,WR,"$8,500,000","$2,400,000",5.90
5128,5129,2022,washington-football-team,Kendall Fuller,CB,"$8,000,000","$3,125,000",5.20


In [58]:
# Clean up the player_salaries DF

# Make cap % a float
player_salaries['cap_percentage'] = player_salaries['cap_percentage'].astype(float)

# Use groupby with idxmax to get the index of rows with the highest cap_percentage per year per team
idx = player_salaries.groupby(['year', 'team'])['cap_percentage'].idxmax()

# Filter the dataframe based on the index
highest_paid = player_salaries.loc[idx].reset_index(drop=True)


# standardize team name to just include city first name
def clean_team_name(team):
    parts = team.split('-')
    city_name = parts[:-1]
    if city_name[-1] == 'football':
        city_name = city_name[:-1]
    return ' '.join(word.title() for word in city_name)

# Apply the custom function to the 'team' column in highest_salary_df
highest_paid['team'] = highest_paid['team'].apply(clean_team_name)

# Display the new DataFrame
highest_paid.head(60)

,index,year,team,name,position,cap_hit,base_salary,cap_percentage
0,1,2020,Arizona,Patrick Peterson,CB,"$12,050,000",-,6.71
1,54,2020,Atlanta,Julio Jones,WR,"$11,200,000","$5,000,000",10.11
2,108,2020,Baltimore,Matt Judon,OLB,"$16,808,000",-,8.49
3,161,2020,Buffalo,Stefon Diggs,WR,"$14,200,000",-,6.70
4,213,2020,Carolina,Teddy Bridgewater,QB,"$8,000,000","$5,000,000",6.69
5,266,2020,Chicago,Khalil Mack,OLB,"$13,300,000","$6,800,000",12.52
6,319,2020,Cincinnati,A.J. Green,WR,"$17,971,000",-,8.72
7,373,2020,Cleveland,Jarvis Landry,WR,"$13,000,000","$1,500,000",6.36
8,425,2020,Dallas,Amari Cooper,WR,"$10,000,000","$2,000,000",5.51
9,477,2020,Denver,Kareem Jackson,SS,"$11,000,000","$2,882,353",6.49


In [73]:
#merge the reg season wins losses with the player salaries df

reg_season_wins.rename(columns={'Team': 'team', 'Year': 'year'}, inplace=True)
# reg_season_wins

# merge dataframes
salaries_with_reg_season_wins_df = pd.merge(highest_paid, reg_season_wins, on=['year', 'team'], how='inner')

salaries_with_reg_season_wins_df

,index,year,team,name,position,cap_hit,base_salary,cap_percentage,Win,Loss,Tie
0,1,2020,Arizona,Patrick Peterson,CB,"$12,050,000",-,6.71,8,8,0
1,54,2020,Atlanta,Julio Jones,WR,"$11,200,000","$5,000,000",10.11,4,12,0
2,108,2020,Baltimore,Matt Judon,OLB,"$16,808,000",-,8.49,11,5,0
3,161,2020,Buffalo,Stefon Diggs,WR,"$14,200,000",-,6.70,13,3,0
4,213,2020,Carolina,Teddy Bridgewater,QB,"$8,000,000","$5,000,000",6.69,5,11,0
...,...,...,...,...,...,...,...,...,...,...,...
79,4917,2022,San Francisco,Jimmie Ward,FS,"$8,900,000","$2,000,000",6.09,13,4,0
80,4970,2022,Seattle,Poona Ford,DE,"$7,900,000","$1,750,000",4.64,9,8,0
81,5023,2022,Tampa Bay,Donovan Smith,LT,"$15,750,000","$2,650,000",8.94,8,9,0
82,5076,2022,Tennessee,Derrick Henry,RB,"$4,000,000","$3,000,000",4.44,7,10,0


In [75]:


# playoff_wins.drop(columns={'Loss', 'Tie'}, inplace=True)

playoff_wins.rename(columns={'Team': 'team', 'Year': 'year', 'Win': 'Playoff Wins'}, inplace=True)

# merge salaries_with_reg_season_wins_df with postseason wins
reg_and_postseason_wins_df = pd.merge(salaries_with_reg_season_wins_df, playoff_wins, on=['year', 'team'], how='left')

reg_and_postseason_wins_df



,index,year,team,name,position,cap_hit,base_salary,cap_percentage,Win,Loss,Tie,Playoff Wins
0,1,2020,Arizona,Patrick Peterson,CB,"$12,050,000",-,6.71,8,8,0,NaN
1,54,2020,Atlanta,Julio Jones,WR,"$11,200,000","$5,000,000",10.11,4,12,0,NaN
2,108,2020,Baltimore,Matt Judon,OLB,"$16,808,000",-,8.49,11,5,0,1.0
3,161,2020,Buffalo,Stefon Diggs,WR,"$14,200,000",-,6.70,13,3,0,2.0
4,213,2020,Carolina,Teddy Bridgewater,QB,"$8,000,000","$5,000,000",6.69,5,11,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79,4917,2022,San Francisco,Jimmie Ward,FS,"$8,900,000","$2,000,000",6.09,13,4,0,2.0
80,4970,2022,Seattle,Poona Ford,DE,"$7,900,000","$1,750,000",4.64,9,8,0,0.0
81,5023,2022,Tampa Bay,Donovan Smith,LT,"$15,750,000","$2,650,000",8.94,8,9,0,0.0
82,5076,2022,Tennessee,Derrick Henry,RB,"$4,000,000","$3,000,000",4.44,7,10,0,NaN


In [78]:
# merge reg_and_postseason_wins_df and superbowl wins
superbowl_wins.columns = ['year', 'team', 'Won Superbowl?']

full_season_wins = pd.merge(reg_and_postseason_wins_df, superbowl_wins, on=['year', 'team'], how='left')

full_season_wins.fillna(0, inplace=True)

full_season_wins.head(60)


,index,year,team,name,position,cap_hit,base_salary,cap_percentage,Win,Loss,Tie,Playoff Wins,Won Superbowl?
0,1,2020,Arizona,Patrick Peterson,CB,"$12,050,000",-,6.71,8,8,0,0.0,0
1,54,2020,Atlanta,Julio Jones,WR,"$11,200,000","$5,000,000",10.11,4,12,0,0.0,0
2,108,2020,Baltimore,Matt Judon,OLB,"$16,808,000",-,8.49,11,5,0,1.0,0
3,161,2020,Buffalo,Stefon Diggs,WR,"$14,200,000",-,6.70,13,3,0,2.0,0
4,213,2020,Carolina,Teddy Bridgewater,QB,"$8,000,000","$5,000,000",6.69,5,11,0,0.0,0
5,266,2020,Chicago,Khalil Mack,OLB,"$13,300,000","$6,800,000",12.52,8,8,0,0.0,0
6,319,2020,Cincinnati,A.J. Green,WR,"$17,971,000",-,8.72,4,11,1,0.0,0
7,373,2020,Cleveland,Jarvis Landry,WR,"$13,000,000","$1,500,000",6.36,11,5,0,1.0,0
8,425,2020,Dallas,Amari Cooper,WR,"$10,000,000","$2,000,000",5.51,6,10,0,0.0,0
9,477,2020,Denver,Kareem Jackson,SS,"$11,000,000","$2,882,353",6.49,5,11,0,0.0,0


In [ ]:
superbowl_wins